In [3]:
import sqlite3
import tkinter as tk
from tkinter import messagebox
import logging
import sys

# ---------------- LOGGING CONFIG ----------------
logging.basicConfig(
    filename="soulsense.log",
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s"
)

logging.info("Application started")

# ---------------- DATABASE SETUP ----------------
try:
    conn = sqlite3.connect("soulsense_db")
    cursor = conn.cursor()

    cursor.execute("""
    CREATE TABLE IF NOT EXISTS scores (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        username TEXT,
        total_score INTEGER
    )
    """)
    conn.commit()

    logging.info("Database initialized successfully")

except Exception:
    logging.error("Database initialization failed", exc_info=True)
    sys.exit(1)

# ---------------- QUESTIONS ----------------
questions = [
    "You can recognize your emotions as they happen.",
    "You manage to stay calm even under pressure.",
    "You are aware of how your emotions affect others.",
    "You can motivate yourself to pursue long-term goals.",
    "You stay optimistic even after facing setbacks.",
    "You are able to express your feelings clearly.",
    "You handle conflict effectively and calmly.",
    "You find it easy to empathize with others' emotions.",
    "You adapt well to changing situations.",
    "You keep your promises and commitments.",
    "You actively listen to others when they speak.",
    "You accept constructive criticism without being defensive.",
    "You remain positive in emotionally difficult situations.",
    "You recognize stress in others and offer support.",
    "You maintain strong relationships over time.",
    "You can identify your emotional triggers.",
    "You take responsibility for your actions.",
    "You resolve misunderstandings diplomatically.",
    "You stay focused and productive when under pressure.",
    "You show appreciation and gratitude regularly."
]

# ---------------- GUI APPLICATION ----------------
class SoulSenseApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Soul Sense EQ Test")
        self.username = ""
        self.current_question = 0
        self.responses = []

        self.create_username_screen()

    def create_username_screen(self):
        self.clear_screen()

        tk.Label(self.root, text="Enter Your Name:", font=("Arial", 14)).pack(pady=20)
        self.name_entry = tk.Entry(self.root, font=("Arial", 14))
        self.name_entry.pack(pady=10)

        tk.Button(self.root, text="Start Test", command=self.start_test, font=("Arial", 12)).pack(pady=20)

    def start_test(self):
        self.username = self.name_entry.get().strip()

        if self.username == "":
            logging.warning("Username input validation failed (empty input)")
            messagebox.showwarning("Input Error", "Please enter your name to start the test.")
            return

        logging.info(f"User session started for username: {self.username}")
        self.show_question()

    def show_question(self):
        self.clear_screen()

        try:
            if self.current_question < len(questions):
                q_text = questions[self.current_question]

                tk.Label(
                    self.root,
                    text=f"Q{self.current_question + 1}: {q_text}",
                    wraplength=400,
                    font=("Arial", 14)
                ).pack(pady=20)

                self.answer_var = tk.IntVar()

                for val, text in enumerate(
                    ["Never (1)", "Sometimes (2)", "Often (3)", "Always (4)"], start=1
                ):
                    tk.Radiobutton(
                        self.root,
                        text=text,
                        variable=self.answer_var,
                        value=val,
                        font=("Arial", 12)
                    ).pack(anchor="w", padx=50)

                tk.Button(
                    self.root,
                    text="Next",
                    command=self.save_answer,
                    font=("Arial", 12)
                ).pack(pady=20)

            else:
                self.finish_test()

        except Exception:
            logging.error("Question loading failed", exc_info=True)
            messagebox.showerror("Error", "An error occurred while loading questions.")

    def save_answer(self):
        ans = self.answer_var.get()

        if ans == 0:
            logging.warning(
                f"Input validation failed: No answer selected (Q{self.current_question + 1})"
            )
            messagebox.showwarning("Input Error", "Please select an answer before proceeding.")
            return

        self.responses.append(
            (self.username, self.current_question + 1, questions[self.current_question], ans)
        )

        self.current_question += 1
        self.show_question()

    def finish_test(self):
        total_score = sum(r[3] for r in self.responses)

        try:
            cursor.execute(
                "INSERT INTO scores (username, total_score) VALUES (?, ?)",
                (self.username, total_score)
            )
            conn.commit()

            logging.info(
                f"Score saved successfully | user={self.username} | score={total_score}"
            )

        except Exception:
            logging.error("Database insert failed while saving final score", exc_info=True)
            messagebox.showerror(
                "Database Error",
                "Unable to save your score. Please try again."
            )
            return

        interpretation = (
            "Excellent Emotional Intelligence!" if total_score >= 65 else
            "Good Emotional Intelligence." if total_score >= 50 else
            "Average Emotional Intelligence." if total_score >= 35 else
            "You may want to work on your Emotional Intelligence."
        )

        self.clear_screen()

        tk.Label(self.root, text=f"Thank you, {self.username}!", font=("Arial", 16)).pack(pady=10)
        tk.Label(
            self.root,
            text=f"Your total EQ score is: {total_score} / 80",
            font=("Arial", 14)
        ).pack(pady=10)
        tk.Label(
            self.root,
            text=interpretation,
            font=("Arial", 14),
            fg="blue"
        ).pack(pady=10)

        tk.Button(self.root, text="Exit", command=self.exit_test, font=("Arial", 12)).pack(pady=20)

    def exit_test(self):
        try:
            conn.close()
            logging.info("Database connection closed")
        except Exception:
            logging.error("Error closing database connection", exc_info=True)

        logging.info("Application exited gracefully")
        self.root.quit()

    def clear_screen(self):
        for widget in self.root.winfo_children():
            widget.destroy()

# ---------------- MAIN LOOP ----------------
if __name__ == "__main__":
    root = tk.Tk()
    root.geometry("500x300")

    app = SoulSenseApp(root)
    root.protocol("WM_DELETE_WINDOW", app.exit_test)

    root.mainloop()
